# Dataset Creation

Convert to python: run `jupyter nbconvert --to python test.ipynb`

In [1]:
import os
import shutil
import wget
import zipfile
import pandas as pd
import datetime
import numpy as np
from glob import glob


## Config

Make sure to change these configs before running the whole notebook.

- Base URL: The BackBlaze dataset URL

In [2]:
base_url = "https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/"

- Base Path: This should be pointing to where to hold the dataset.  
Note that this relative path is relative to the current working directory, or in other words, usually where this ipynb is.

In [3]:
base_path = os.path.dirname("../../HDD_dataset/")

- Years: Years of data to download

In [4]:
# years = ["2013", "2014", "2015", "2016", "2017", "2018", "2019"]
years = ["2013", "2016"]

- Model: The specific HDD model we want to keep the data for

In [5]:
model = "ST3000DM001"

- Find Failed: if `True`, keep only failed HDDs, otherwise keep all HDDs

In [6]:
find_failed = False

zips contain different directory names or no directory at all, which causes unavoidable "spaghettiness" in the code.
For example, the data in year 2013 is in the directory "2013", so the key is `"2013"`. The data from 2016 are in the root of the zip file, hence the key is `None`. 

In [7]:
# zips contain different directory names or no directory at all, which causes
# unavoidable "spaghettiness" in the code
suffixes = {
    "data_2013.zip": '2013',
    "data_2014.zip": '2014',
    "data_2015.zip": '2015',
    "data_Q1_2016.zip": None,
    "data_Q2_2016.zip": None,
    "data_Q3_2016.zip": None,
    "data_Q4_2016.zip": None,
    "data_Q1_2017.zip": None,
    "data_Q2_2017.zip": None,
    "data_Q3_2017.zip": None,
    "data_Q4_2017.zip": None,
    "data_Q1_2018.zip": None,
    "data_Q2_2018.zip": None,
    "data_Q3_2018.zip": None,
    "data_Q4_2018.zip": None,
    "data_Q1_2019.zip": None,
    "data_Q2_2019.zip": None,
    "data_Q3_2019.zip": None,
}

## Get Dataset

Download and unzip the dataset, moving files to the correct directory
The dataset is structured as follows:
```
base_path  
├── 2018
│   ├── 2018-01-01.csv
│   ├── 2018-01-02.csv
│   └── 2018-01-03.csv
├── 2019
│   ├── 2019-01-01.csv
│   ├── 2019-01-02.csv
│   └── 2019-01-03.csv
├── data_Q1_2018.zip
├── data_Q1_2019.zip
├── data_Q2_2018.zip
├── data_Q2_2019.zip
├── data_Q3_2018.zip
├── data_Q3_2019.zip
└── data_Q4_2018.zip
```

In [8]:
os.makedirs(base_path, exist_ok=True)
# just in case they are passed as int
years = [str(_) for _ in years]
for y in years:
    print("Year:", y)
    year_path = os.path.join(base_path, y)
    os.makedirs(year_path, exist_ok=True)
    for zip_name, unzip_dir in suffixes.items():
        if y in zip_name:
            url = base_url + zip_name
            zip_path = os.path.join(base_path, zip_name)
            if not os.path.exists(zip_path):
                print("Downloading:", url)
                wget.download(url, out=base_path)
            print("\nUnzipping:", zip_path)
            dest_path = year_path if unzip_dir is None else base_path
            with zipfile.ZipFile(zip_path, 'r') as z:
                z.extractall(dest_path)

            if unzip_dir is not None and unzip_dir != y:
                unzip_path = os.path.join(dest_path, unzip_dir)
                for f in os.listdir(unzip_path):
                    shutil.move(os.path.join(unzip_path, f),
                            os.path.join(year_path, f))
                os.rmdir(unzip_path)


Year: 2013

Unzipping: ../../HDD_dataset/data_2013.zip
Year: 2016

Unzipping: ../../HDD_dataset/data_Q1_2016.zip

Unzipping: ../../HDD_dataset/data_Q2_2016.zip

Unzipping: ../../HDD_dataset/data_Q3_2016.zip

Unzipping: ../../HDD_dataset/data_Q4_2016.zip


## Collect all serial numbers of given HDD model

In [9]:
# Define the directories for each year
year_dirs = {str(year): os.path.join(base_path, str(year)) for year in range(2013, 2020)}

list_failed = []

# Create temp directory
temp_dir = os.path.join(base_path, 'temp')
os.makedirs(temp_dir, exist_ok=True)

# for each year
for year in years:
    year_dir = year_dirs[year]
    files = glob(os.path.join(year_dir, '*.csv'))

    # for each file, or day
    for file_path in sorted(files):
        try:
            file_r = pd.read_csv(file_path)
        except FileNotFoundError:
            print(f"Error: The file {file_path} does not exist.")
            continue

        # choose the HDD model we need
        model_chosen = file_r[file_r['model'] == model]

        # if that particular HDD model is not present, continue
        if model_chosen.empty:
            continue

        # Print processing day
        # print('processing day ' + str(model_chosen['date'].values))

        if find_failed:
            # choose only the failed hard drives
            model_chosen = model_chosen[model_chosen['failure'] == 1]
            # print(f"Number of entries after filtering by failure: {len(model_chosen)}")

        # keep the failed hard drives' serial number
        list_failed.extend(model_chosen['serial_number'].values)

# Save the list of failed or all hard drives
suffix = 'failed' if find_failed else 'all'
# np.save(os.path.join(temp_dir, f'HDD_{suffix}_{model}'), list_failed)

## Read to a DataFrame

In [10]:
failed = list_failed
# failed = set(np.load(os.path.join(temp_dir, f'HDD_all_{model}.npy')))

database = pd.DataFrame()

# Iterate over each year
for year in years:
    year_path = year_dirs[year]
    files = sorted([f for f in os.listdir(year_path) if f.endswith('.csv')])

    # Iterate over each file in the directory
    for file in files:
        file_path = os.path.join(year_path, file)
        file_date = datetime.datetime.strptime(file.split('.')[0], '%Y-%m-%d')
        old_time = datetime.datetime.strptime(f'{year}-01-01', '%Y-%m-%d')
        
        if file_date >= old_time:
            df = pd.read_csv(file_path)
            model_chosen = df[df['model'] == model]
            relevant_rows = model_chosen[model_chosen['serial_number'].isin(failed)]

            # Drop unnecessary columns since the following columns are not standard for all models
            drop_columns = [col for col in relevant_rows if 'smart_' in col and int(col.split('_')[1]) in {22, 220, 222, 224, 226}]
            relevant_rows.drop(columns=drop_columns, errors='ignore', inplace=True)

            # Append the row to the database
            database = pd.concat([database, relevant_rows], ignore_index=True)
            print('adding day ' + str(model_chosen['date'].values))

# Save the database to a pickle file
# database.to_pickle(os.path.join(temp_dir, f'All_failed_appended_{model}.pkl'))

adding day ['2013-04-10' '2013-04-10' '2013-04-10' ... '2013-04-10' '2013-04-10'
 '2013-04-10']
adding day ['2013-04-11' '2013-04-11' '2013-04-11' ... '2013-04-11' '2013-04-11'
 '2013-04-11']
adding day ['2013-04-12' '2013-04-12' '2013-04-12' ... '2013-04-12' '2013-04-12'
 '2013-04-12']
adding day ['2013-04-13' '2013-04-13' '2013-04-13' ... '2013-04-13' '2013-04-13'
 '2013-04-13']
adding day ['2013-04-14' '2013-04-14' '2013-04-14' ... '2013-04-14' '2013-04-14'
 '2013-04-14']
adding day ['2013-04-15' '2013-04-15' '2013-04-15' ... '2013-04-15' '2013-04-15'
 '2013-04-15']
adding day ['2013-04-16' '2013-04-16' '2013-04-16' ... '2013-04-16' '2013-04-16'
 '2013-04-16']
adding day ['2013-04-17' '2013-04-17' '2013-04-17' ... '2013-04-17' '2013-04-17'
 '2013-04-17']
adding day ['2013-04-18' '2013-04-18' '2013-04-18' ... '2013-04-18' '2013-04-18'
 '2013-04-18']
adding day ['2013-04-19' '2013-04-19' '2013-04-19' ... '2013-04-19' '2013-04-19'
 '2013-04-19']
adding day ['2013-04-20' '2013-04-20' '2

## Assemble the data for later consumption  

Select Serial numbers and the dates they presents in the dataset

In [11]:
# database = pd.read_pickle('../temp/All_failed_appended_' + model +'.pkl')
base = pd.DataFrame(database.groupby('serial_number')['date'].apply(list))
base

,date
serial_number,
S1F006KC,"[2013-04-10, 2013-04-11, 2013-04-12, 2013-04-1..."
S1F006XG,"[2013-04-10, 2013-04-11, 2013-04-12, 2013-04-1..."
S1F007YW,"[2013-04-10, 2013-04-11, 2013-04-12, 2013-04-1..."
S1F009LV,"[2013-04-10, 2013-04-11, 2013-04-12, 2013-04-1..."
S1F009X6,"[2013-06-06, 2013-06-07, 2013-06-08, 2013-06-0..."
...,...
Z1F26YZB,"[2013-04-10, 2013-04-11, 2013-04-12, 2013-04-1..."
Z1F282ZV,"[2013-04-10, 2013-04-11, 2013-04-12, 2013-04-1..."
Z1F2BHER,"[2013-10-15, 2013-10-16, 2013-10-17, 2013-10-1..."


Assemble the DataFrame

In [12]:
data_input = os.path.join(base_path, "data_input")
os.makedirs(data_input, exist_ok=True)

for i,smart in enumerate(database.keys()[4:]):
	# print('concatenating feature ' + str(i) + '/'+ str(database.keys()[2:].shape[0]))
	base = pd.concat([base, database.groupby('serial_number')[smart].apply(list)], axis=1)

Pickle the DataFrame

In [13]:
filename = f"2013_2014_2015_2016_2017_{'failed' if find_failed else 'all'}_{model}.pkl"
base.to_pickle(os.path.join(data_input, filename))


In [14]:
base
# TODO: find bad drives, plot 1 smart data on y-axis against date on x-axis

,date,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
serial_number,,,,,,,,,,,,,,,,,,,,,
S1F006KC,"[2013-04-10, 2013-04-11, 2013-04-12, 2013-04-1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[121883848.0, 162064440.0, 128194248.0, 196566...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
S1F006XG,"[2013-04-10, 2013-04-11, 2013-04-12, 2013-04-1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[227423400.0, 237121256.0, 2430864.0, 12126704...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
S1F007YW,"[2013-04-10, 2013-04-11, 2013-04-12, 2013-04-1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[99588288.0, 120083136.0, 220986728.0, 2899856...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
S1F009LV,"[2013-04-10, 2013-04-11, 2013-04-12, 2013-04-1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[57952968.0, 183862160.0, 135759040.0, 2283659...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, 